In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [ ]:
import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation, PReLU, LeakyReLU
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
from keras.wrappers.scikit_learn import KerasClassifier
from scipy.stats import randint as sp_randint
from sklearn.model_selection import ParameterSampler
import time
from keras.callbacks import History 
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import warnings

# deal with potential warnings
warnings.filterwarnings(action='ignore')

read processed array data, usually takes 1 or 2 minutes.

In [ ]:
data1 = np.load('data/data1.npy')
data2 = np.load('data/data2.npy')
data3 = np.load('data/data3.npy')
data4 = np.load('data/data4.npy')
data = np.concatenate((data1,data2,data3,data4))
x_test = np.load('data/testdata.npy')
testID = np.load('data/testdataID.npy')

x_test /= 255

train/test split and normalization

In [ ]:
# train/test ratio
ratio = 0.8

# shuffle dataset
seed(1)
np.random.shuffle(data)
x_train = []
y_train = []
x_dev = []
y_dev = []
for index,item in enumerate(data):
    if index<len(data)*ratio:
        x_train.append(item[0])
        y_train.append(item[1])
    else:
        x_dev.append(item[0])
        y_dev.append(item[1])
    
x_train = np.asarray(x_train)
x_dev = np.asarray(x_dev)
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)
x_test = np.asarray(x_test)

# normalize input
x_train /= 255
x_dev /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 2)[:,-1]
y_dev = keras.utils.to_categorical(y_dev, 2)[:,-1]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print('x_dev shape:', x_dev.shape)
print(x_dev.shape[0], 'validtion samples')
print('x_test shape:', x_test.shape)

In [ ]:
np.save('data/save_data', (x_train, y_train, x_dev, y_dev))

In [ ]:
(x_train, y_train, x_dev, y_dev) = np.load('data/save_data.npy')

CNN architecture

In [ ]:
def create_model(lr=0.001, dropout_rate=[0.5], alpha=0.0001):
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), strides = (1, 1), input_shape=(64, 64, 3), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(16, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(32, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(32, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(64, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(Conv2D(128, (3, 3), strides = (1, 1), border_mode='same', W_regularizer=regularizers.l2(alpha)))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#     model.add(GlobalMaxPooling2D())

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(PReLU())
    model.add(Dropout(dropout_rate[0]))
    model.add(Dense(512))
    model.add(PReLU())
    model.add(Dropout(dropout_rate[1]))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=lr, amsgrad=True),
                  metrics=['accuracy'])
        
    return model

Training process

In [ ]:
# callbacks, early-stopping
callbacks = keras.callbacks.ModelCheckpoint('prelu_model_1.h5', 
                                             monitor='val_acc', verbose=1, save_best_only=True, 
                                             save_weights_only=False, mode='auto', period=1)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

callback_list = [callbacks]

# hyper-parameters
param = {'lr':[0.001],
#          'batch_size':np.power(2,np.arange(5,7)),
         'dropout_rate':np.round(np.arange(0.4,0.9,0.1),1)}

param_list = list(ParameterSampler(param, n_iter=5))
rounded_list = [dict((k, v) for (k, v) in d.items()) for d in param_list]
print(rounded_list)

In [ ]:
# sklearn wrapper training
model = KerasClassifier(build_fn=create_model, verbose=1, 
                        epochs=100,
                        lr=0.001,
                        batch_size=64,
                        dropout_rate=[0.7],
                        alpha=0.01,
                        validation_data=(x_dev,y_dev),
                        callbacks=callback_list)
train_history = model.fit(x_train, y_train)
#     model.model.save('my_model_1.h5')

In [ ]:
# continue training
temp_weights = [layer.get_weights() for layer in new_model.layers]
model = create_model(lr=0.0001,
                     dropout_rate=[0.7],
                     alpha=0.01)
for j in range(len(temp_weights)):
    model.layers[j].set_weights(temp_weights[j])
    
train_history = model.fit(x_train, y_train, verbose=1,
                          batch_size=64,
                          validation_data=(x_dev,y_dev), 
                          epochs=100,
                          callbacks=callback_list)

In [ ]:
temp_weights = [layer.get_weights() for layer in new_model.layers]

model = create_model(lr=1e-6,
                     dropout_rate=[0.7, 0.7],
                     alpha=0.01)

for j in range(len(temp_weights)):
    model.layers[j].set_weights(temp_weights[j])

datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             rotation_range=20,
                             horizontal_flip=True)

datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), 
                    verbose=1,
                    steps_per_epoch=len(x_train) // 32,
                    validation_data=(x_dev,y_dev), 
                    epochs=100,
                    callbacks=callback_list)

In [ ]:
model.save('prelu_model_1.h5')

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

y_pre = np.round(new_model.predict(x_dev, verbose=1))
confusion_matrix(y_dev, y_pre)

evaluation

In [ ]:
%matplotlib inline
plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])
plt.grid(True)
plt.show()

offline working

In [ ]:
new_model = load_model('prelu_model.h5')
new_model.model.summary()

In [ ]:
# predict on test set 
score = np.round(new_model.predict(x_test, verbose=1)).astype(np.int32)

In [ ]:
sub = pd.DataFrame()
sub['id'] = pd.Series(testID)
sub['label'] = score
sub.to_csv('DvCsubmission.csv', index = False)

Try some other fur balls!

In [ ]:
img_path = 'pics/wechat.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img) / 255
plt.imshow(x)
plt.show()
x = np.expand_dims(x, axis=0)
# print(model.predict(x))
print(['cat', 'dog'][int(new_model.predict(x))])
print(new_model.predict(x))